In [68]:
with open("input17.txt", "r") as file:
    A = int(file.readline().split(":")[1])
    B = int(file.readline().split(":")[1])
    C = int(file.readline().split(":")[1])
    file.readline()
    program = [int(x) for x in (file.readline().split(":")[1]).split(",")]

def combo(operand):
    if operand in {0,1,2,3}: return operand
    if operand==4: return A
    if operand==5: return B
    if operand==6: return C

i, out = 0, []
while i < len(program):
    opcode, operand = program[i], program[i+1]
    i+=2
    if opcode==0: A = A//(2**combo(operand))
    if opcode==1: B = B^operand
    if opcode==2: B = combo(operand)%8
    if opcode==3:
        if A!=0: i=operand
    if opcode==4: B = B^C
    if opcode==5: out.append(combo(operand)%8)
    if opcode==6: B = A//(2**combo(operand))
    if opcode==7: C = A//(2**combo(operand))
",".join(str(n) for n in out)

'4,3,7,1,5,3,0,5,4'

Part II

Brute force no good. Nothing up to 39122204

For the program to close, A has to be 0, else the last instruction (3,0) will restart the program.<br> What affects A? Only the instruction (0,3), which divides it by 8. Hopefully, A is not affected by anyone else. So we can quickly get an estimate of the size of the output (there's one output per cycle) from the value of A.

We want a 16-long output, so we need an A of the form $[8^{15}, 8^{16})$

In [26]:
8**15, 8**16

(35184372088832, 281474976710656)

Thinking in base 8, each round A loses its least significant digit.

Let's think of the last number printed out. It's generated when A has been reduced to 1 digit, its originally most significant one (msd). What can that digit be for the last number printed out to be 0?

In [5]:
program = [int(x) for x in "0,3,5,4,3,0".split(",")]
program = [int(x) for x in "2,4,1,2,7,5,4,5,0,3,1,7,5,5,3,0".split(",")]

for A in range(8):
    B = A%8
    B = B^2
    C = A//(2**B)
    B = B^C
    B = B^7
    # A = A//8
    print(f"A={A}, out={B%8}")

A=0, out=5
A=1, out=4
A=2, out=5
A=3, out=7
A=4, out=1
A=5, out=0
A=6, out=3
A=7, out=2


So the first digit has to be 5 (in base 8). Thus we can get the second digit, and so on

In [49]:
program = [int(x) for x in "2,4,1,2,7,5,4,5,0,3,1,7,5,5,3,0".split(",")]

Alist = [] #from most to least significant base 8 digits
for j in range(len(program)): #j-th digit
    for A_lsd in range(8): #current lsd of A
        A = 8*sum(8**i*Alist[::-1][i] for i in range(len(Alist))) + A_lsd #inserting the new digit to the right
        B = (A%8)^2
        C = A//(2**B)
        B = B^C
        B = B^7
        # A = A//8 #this is done by the list construction
        if B%8 == program[-j-1]:
            Alist.append(A_lsd)
            break
Alist, sum(8**i*Alist[::-1][i] for i in range(len(Alist)))

([5, 3, 2, 2, 3, 5, 3, 7, 2, 7, 2, 3, 6, 0, 1, 7], 190384615275535)

Straight in numerical form, and getting all valid As

In [13]:
program = [int(x) for x in "2,4,1,2,7,5,4,5,0,3,1,7,5,5,3,0".split(",")]

allA = [0]
for j in range(len(program)): #j-th digit
    newA = []
    for Asofar in allA:
        for A_lsd in range(8): #current lsd of A
            A = 8*Asofar + A_lsd #inserting the new digit to the right
            B = (A%8)^2
            C = A//(2**B)
            B = B^C
            B = B^7
            # A = A//8 #doing this backwards
            if B%8 == program[-j-1]:
                newA.append(A)
    allA = newA
allA

[190384615275535, 190384625499151, 207976801319951, 207976811543567]

In [6]:
for A in allA:
    out = []
    while A!=0:
        B = A%8
        B = B^2
        C = A//(2**B)
        B = B^C
        B = B^7
        A = A//8
        out.append(B%8)
    print(out, out == program)

[2, 4, 1, 2, 7, 5, 4, 5, 0, 3, 1, 7, 5, 5, 3, 0] True
[2, 4, 1, 2, 7, 5, 4, 5, 0, 3, 1, 7, 5, 5, 3, 0] True
[2, 4, 1, 2, 7, 5, 4, 5, 0, 3, 1, 7, 5, 5, 3, 0] True
[2, 4, 1, 2, 7, 5, 4, 5, 0, 3, 1, 7, 5, 5, 3, 0] True


It's a bit custom made. For the example, it goes like:

In [28]:
program = [int(x) for x in "0,3,5,4,3,0".split(",")]
#because this one starts by div by 8, the lsd is irrelevant

Alist = [] #from most to least significant base 8 digits
for j in range(len(program)): #j-th digit
    for A_lsd in range(8): #current lsd of A
        # A = A//8 #this is done by the list construction
        A = 8*sum(8**i*Alist[::-1][i] for i in range(len(Alist))) + A_lsd
        if A%8 == program[-j-1]:
            Alist.append(A_lsd)
            break
Alist.append(0) #0 is the lowest possible lsd
Alist, sum(8**i*Alist[::-1][i] for i in range(len(Alist)))

([0, 3, 4, 5, 3, 0, 0], 117440)

In [29]:
A, out = 117440, []
while A!=0:
    A = A//8
    out.append(A%8)
out, out == program

([0, 3, 5, 4, 3, 0], True)

In [8]:
program = [int(x) for x in "0,3,5,4,3,0".split(",")]

allA = [0]
for j in range(len(program)): #j-th digit
    newA = []
    for Asofar in allA:
        for A_lsd in range(8): #current lsd of A
            A = 8*Asofar + A_lsd #inserting the new digit to the right
            if A%8 == program[-j-1]:
                newA.append(A)
    allA = newA
allA = [8*x+i for x in allA for i in range(8)]
allA

[117440, 117441, 117442, 117443, 117444, 117445, 117446, 117447]

This smaller could be just brute forced:

In [11]:
for Ainit in range(8**5,8**6):
    A, out = Ainit, []
    while A!=0:
        A = A//8
        out.append(A%8)
    if out == program:
        print(Ainit, out)

117440 [0, 3, 5, 4, 3, 0]
117441 [0, 3, 5, 4, 3, 0]
117442 [0, 3, 5, 4, 3, 0]
117443 [0, 3, 5, 4, 3, 0]
117444 [0, 3, 5, 4, 3, 0]
117445 [0, 3, 5, 4, 3, 0]
117446 [0, 3, 5, 4, 3, 0]
117447 [0, 3, 5, 4, 3, 0]
